In [1]:
from test_framework.messages import *
from test_framework.p2p import *
import socket
import binascii
import socket
import struct
import codecs
import time
import random
import hashlib
from datetime import datetime
import io

#### Custom methods from https://github.com/bitcoin/bitcoin/tree/master/test/functional/test_framework

#### Settings

In [2]:
default_port    = 18333
magic_          = 0x0709110B
max_n_bits      = 0x1d00ffff
version_        = 70015

buff_size       = 0x02000000  # https://github.com/bitcoin/bitcoin/blob/60abd463ac2eaa8bc1d616d8c07880dc53d97211/src/serialize.h#L23
 
command         = b"version"
#host            = "8.9.3.218"
host            = "82.197.160.8"   # node Thush
MAGIC_BYTES     = b"\x0b\x11\x09\x07"

In [3]:
class Message:
    __slots__ = ("addrFrom", "nPortFrom", "addrTo", "nPortTo", "nNonce", "relay", "nServices",
                 "nStartingHeight", "nTime", "nVersion", "strSubVer","bMagic", "bCommand","timestamp","with_payload")
    
    def __init__(self, bCommand = b'undefined', addrTo = '0.0.0.0', nPortTo = 0, bMagic= 0x0709110B, nVersion= 70015, nServices= 0,with_payload=True):
        self.nNonce = random.getrandbits(64)
        self.nStartingHeight = 596306
        self.relay = False
        self.nPortFrom = 0
        self.addrFrom = ""
        self.bCommand = bCommand
        self.addrTo = addrTo
        self.nPortTo = nPortTo
        self.with_payload = with_payload
        self.bMagic = bMagic
        self.nVersion = nVersion
        self.nServices  = nServices

    def serialize(self):
        # header 
        header     = b""
        header     += struct.pack("i",self.bMagic)                    # magic value                           
        header     += struct.pack("<12s",self.bCommand)                     # command
        
        # payload
        payload    = b""
        if self.with_payload:
            payload    += struct.pack("i", self.nVersion)
            payload    += struct.pack("Q", self.nServices)
            payload    += struct.pack("q", int(time.time()))
            payload    += struct.pack("Q",0);

            payload    +=b"\x00" * 10 + b"\xff" * 2
            payload    += socket.inet_aton(self.addrTo)
            payload    += struct.pack(">H",self.nPortTo)                    # Receiver Port
            payload    += struct.pack("Q",0)
            payload    += struct.pack(">16s",bytes(self.addrFrom,'utf-8'))  # Sender IP Address
            payload    += struct.pack(">H",self.nPortFrom)                  # Sender Port
            payload    += struct.pack("<Q", self.nNonce)
            payload    += struct.pack("B",0)                                # Bytes in version string
            payload    += struct.pack("i", self.nStartingHeight)
            payload    += struct.pack("?", self.relay)
        
        # length and checksum
        length     = struct.pack("I", len(payload));
        checksum   = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4];
        
        return header + length + checksum + payload
    
    def deserialize(self, reply):
        if reply.getbuffer().nbytes == 0:
            return
        self.bMagic          = struct.unpack("i",reply.read(4))[0]
        self.bCommand        = struct.unpack("<12s",reply.read(12))[0].split(b"\x00", 1)[0]
        length               = struct.unpack("I", reply.read(4))[0]
        checksum             = struct.unpack("I", reply.read(4))[0]
        
        if self.bCommand == b'addr':
            print(f"Could be response to getaddr. Remaining buffer size: {reply.getbuffer().nbytes}")
            #print(binascii.hexlify(reply.getbuffer()))
            self.decode_addr(reply)
        
        print(f"length: {length}")
        if length > 0:
            self.nVersion        = struct.unpack("i", reply.read(4))[0]
            self.nServices       = struct.unpack("Q", reply.read(8))[0]
            self.timestamp       = datetime.fromtimestamp(struct.unpack("q", reply.read(8))[0])
            toServices           = struct.unpack("Q", reply.read(8))[0]
            # Skip 12 Bytes for IPv4
            reply.read(12)
            self.addrTo          = socket.inet_ntoa(reply.read(4))
            self.nPortTo         = struct.unpack(">H", reply.read(2))[0]
            
            fromServices         = struct.unpack("Q", reply.read(8))[0]
            # Skip 12 Bytes for IPv4
            reply.read(12)
            self.addrFrom        = socket.inet_ntoa(reply.read(4))
            self.nPortFrom       = struct.unpack(">H", reply.read(2))[0]
            self.nNonce          = struct.unpack("<Q", reply.read(8))[0]
            user_agent_bytes     = struct.unpack("B", reply.read(1))[0]
            self.nStartingHeight = struct.unpack("i",reply.read(4))
            relay                = struct.unpack("?",reply.read(1))
        
    def decode_addr(self, reply):
        i = 0;
        print(binascii.hexlify(reply.read(3)))
        
        while(reply.getbuffer().nbytes > 0):
            if i > 300:
                break
            i = i+1
            
            #print(binascii.hexlify(reply.read(30)))
            #continue
            
            time = reply.read(4)
            if time == b'':
                break
            services = reply.read(8)
            reply.read(12)
            ip =  socket.inet_ntoa(reply.read(4))
            port = struct.unpack(">H", reply.read(2))[0]
            
            print(f"{ip}:{port}")     

In [4]:
message            = Message()
message.nVersion   = 70015
message.nServices  = 0
message.addrTo     = "82.197.160.8"

message.nPortTo    = 18333

message.bMagic     = magic_

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM);

reply_1 = Message()
reply_2 = Message()
reply_3 = Message()

sock.connect((message.addrTo,message.nPortTo));

sock.send(message.serialize(b'version'));
print(f"Sending {message.bCommand}")
time.sleep(1);
reply1 = sock.recv(buff_size);
reply_1.deserialize(io.BytesIO(reply1))
print(f"Received {reply_1.bCommand}")

sock.close()

TypeError: serialize() takes 1 positional argument but 2 were given

In [ ]:
import asyncio

async def tcp_echo_client(messages):
    print("Started!")
    
    message = await messages.get()
    
    print(message)
    reader, writer = await asyncio.open_connection(
        message.addrTo, message.nPortTo,limit=buff_size)

    print(f'Send: {message.bCommand}')
    writer.write(message.serialize())
    
    print("Message sent")
    
    data = await reader.read(10) # Problem here
    print(f'Received: {data.decode()!r}')

    print('Close the connection')
    writer.close()

#asyncio.run(tcp_echo_client(messages)) no necessary because jupyter

#test = tcp_echo_client(messages)

In [ ]:
messages = asyncio.Queue() # FIFO
messages.put_nowait(Message(bCommand=b'version', addrTo=host,nPortTo=default_port))


In [ ]:
messages = asyncio.Queue() # FIFO
messages.put_nowait(Message(bCommand=b'version', addrTo=host,nPortTo=default_port))
loop = asyncio.get_event_loop()
asyncio.run_coroutine_threadsafe(tcp_echo_client(messages), loop)

In [5]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM);

reply_1 = Message()
reply_2 = Message()
reply_3 = Message()
reply_4 = Message()

sock.connect((message.addrTo,message.nPortTo));

message.bCommand = b'version'

sock.send(message.serialize());
print(f"Sending {message.bCommand}")
time.sleep(1);
reply1 = sock.recv(buff_size);
reply_1.deserialize(io.BytesIO(reply1))
print(f"Received {reply_1.bCommand}")
print(f"Receiver Version: {reply_1.nVersion}")

message.bCommand = b'verack'
sock.send(message.serialize());
print(f"Sending {message.bCommand}")
time.sleep(1);
reply2 = sock.recv(buff_size);
reply_2.deserialize(io.BytesIO(reply2))
print(f"Received {reply_2.bCommand}")

message.bCommand = b'getaddr'
message.with_payload = False
sock.send(message.serialize());
print(f"Sending {message.bCommand}")
time.sleep(0);
reply3 = sock.recv(buff_size);
reply_3.deserialize(io.BytesIO(reply3))
print(f"Received {reply_3.bCommand}")

#sock.send(message.serialize(b'getaddr',with_payload=False));
#print(f"Sending {message.bCommand}")
time.sleep(0);
reply4 = sock.recv(buff_size);
reply_4.deserialize(io.BytesIO(reply4))
print(f"Received {reply_4.bCommand}")

sock.close()

Sending b'version'
length: 102
Received b'version'
Receiver Version: 70016
Sending b'verack'
length: 0
Received b'sendheaders'
Sending b'getaddr'
Could be response to getaddr. Remaining buffer size: 1448
b'fde803'
54.169.173.79:18333
0.0.0.1:18333
180.244.137.239:18333
102.91.5.130:18333
107.161.154.26:18333
182.66.177.255:18333
213.194.124.194:18333
102.91.5.169:18333
88.77.163.26:18333
178.234.199.83:18333
185.34.33.2:19332
203.132.94.196:18333
183.23.73.24:18333
197.210.226.62:18333
120.138.12.237:18333
94.19.235.41:28333
210.56.97.197:18333
83.142.237.197:18333
212.47.242.83:18333
190.237.60.12:18333
178.63.2.220:18333
90.140.133.63:18333
46.114.142.180:18333
59.201.15.44:18333
205.185.117.149:18333
94.231.253.18:18333
52.198.65.143:18333
94.130.207.84:8333
84.2.155.88:18333
145.185.205.165:18333
155.17.105.0:18333
154.31.112.76:18333
90.1.249.212:39388
0.0.0.1:18333
159.203.183.112:18333
80.92.232.39:40263
39.118.118.126:18333
102.89.34.36:18333
52.204.94.157:18333
18.116.41.169:1

OSError: packed IP wrong length for inet_ntoa

In [ ]:
binascii.hexlify(reply3)

reply_3 = Message()

reply_3.deserialize(io.BytesIO(reply3))

In [30]:
import asyncio
import queue

default_port    = 18333
host            = "82.197.160.8"   # node Thush

class SimpleGetAddrClient(asyncio.Protocol):
    def __init__(self, messages, on_con_lost):
        self.messages = messages
        self.on_con_lost = on_con_lost
        self.transport = None
        self.recvbuf = b""

    def connection_made(self, transport):
        self.transport = transport
        message = self.messages.get()
        transport.write(message.serialize())

    def data_received(self, data):
        if(len(data) > 0):
            self.recvbuf += data
            self.on_data()
        
    def on_data(self):
        try:
            while True:
                if len(self.recvbuf) < 4:
                    return
                if self.recvbuf[:4] != self.magic_bytes:
                    raise ValueError("magic bytes mismatch: {} != {}".format(repr(self.magic_bytes), repr(self.recvbuf)))
                if len(self.recvbuf) < 4 + 12 + 4 + 4:
                    return
                    msgtype = self.recvbuf[4:4+12].split(b"\x00", 1)[0]
                msglen = struct.unpack("<i", self.recvbuf[4+12:4+12+4])[0]
                checksum = self.recvbuf[4+12+4:4+12+4+4]
                if len(self.recvbuf) < 4 + 12 + 4 + 4 + msglen:
                    return
                msg = self.recvbuf[4+12+4+4:4+12+4+4+msglen]
                th = sha256(msg)
                h = sha256(th)
                if checksum != h[:4]:
                    raise ValueError("got bad checksum " + repr(self.recvbuf))
                self.recvbuf = self.recvbuf[4+12+4+4+msglen:]
                if msgtype not in MESSAGEMAP:
                    raise ValueError("Received unknown msgtype from %s:%d: '%s' %s" % (self.dstaddr, self.dstport, msgtype, repr(msg)))
                f = BytesIO(msg)
                t = MESSAGEMAP[msgtype]()
                t.deserialize(f)
                self._log_message("receive", t)
                self.on_message(t)
        except Exception as e:
            logger.exception('Error reading message:', repr(e))
            raise

    def connection_lost(self, exc):
        print('The server closed the connection')
        self.on_con_lost.set_result(True)


async def main():
    # Get a reference to the event loop as we plan to use
    # low-level APIs.
    loop = asyncio.get_running_loop()

    on_con_lost = loop.create_future()

    transport, protocol = await loop.create_connection(
        lambda: SimpleGetAddrClient(send_queue, on_con_lost),
        host, default_port)

    # Wait until the protocol signals that the connection
    # is lost and close the transport.
    try:
        await on_con_lost
    finally:
        transport.close()

        
send_queue = queue.Queue() # FIFO
send_queue.put(Message(bCommand=b'version', addrTo=host,nPortTo=default_port))
send_queue.put(Message(bCommand=b'verack', addrTo=host,nPortTo=default_port))
send_queue.put(Message(bCommand=b'getaddr', addrTo=host,nPortTo=default_port,with_payload=False))

loop = asyncio.get_event_loop()
future = asyncio.run_coroutine_threadsafe(main(), loop)

In [15]:
reply = binascii.unhexlify(b'0b11090776657273696f6e00000000006600000088239f878011010009040000000000001822386200000000000000000000000000000000000000000000ffffd537f672442009040000000000000000000000000000000000000000000000002565d8600e262ad9102f5361746f7368693a32322e302e302f956d2100010b11090776657261636b000000000000000000005df6e0e2')

In [16]:
msg = Message()
msg.deserialize(io.BytesIO(reply))
msg.bCommand

length: 102


b'version'

b'0b110907696e7600000000000000000025000000d30260160102000000c738c808ebae6c5e036175fa725d54e156abf63772a565269b00000000000000'
b'0b110907696e7600000000000000000025000000d30260160102000000c738c808ebae6c5e036175fa725d54e156abf63772a565269b00000000000000'
b'0b1109076164647200000000000000001f000000838d8457014d213862080400000000000000000000000000000000ffff6dfca0f8479d'
